In [1]:
#warcio is used to iterate over warc file
#boilerpipe is used to extract text from HTML pages

from warcio.archiveiterator import ArchiveIterator
import io, time, json
from boilerpipe.extract import Extractor
import datetime
from collections import defaultdict
import numpy as np
from scipy import stats
import json
from pathlib import Path

In [3]:
#RUN EVERYTHING
a = datetime.datetime.now()
print (a)
conversion = 0
    

path = 'jan_2018_paths/' 
wet = 'CC-MAIN-20180116070444-20180116090444-00000.warc.wet'
warc = 'CC-MAIN-20180116070444-20180116090444-00000.warc'
with open(path+wet, 'rb') as wet, open(path+warc, 'rb') as warc:
    warc_data_file = {}
    wet_data_file = {}
    
    web_pages = []
    not_parsed_types = defaultdict(int)
    warc_offset=0
    wet_offset=0
    parsed_pages = 0
    not_parsed_pages = 0
    
    iter_warc = ArchiveIterator(warc)
    for wet_record in ArchiveIterator(wet):
        warc_record = iter_warc.__next__()
        wet_offset+=1
        warc_offset+=1
        
        #there is only one 'warcinfo' in warc files
        #represents the metadata of that file
        if wet_record.rec_type == 'warcinfo':
            
            #warc_warcinfo header
            tmp_wet = dict(wet_record.rec_headers.headers)
            wet_data_file['date']=tmp_wet['WARC-Date']
            wet_data_file['filename']=tmp_wet['WARC-Filename']
            
            tmp_warc = dict(warc_record.rec_headers.headers)
            warc_data_file['date']=tmp_warc['WARC-Date']
            warc_data_file['filename']=tmp_warc['WARC-Filename']
            
            
        if wet_record.rec_type == 'conversion':
            
            #record warc_response header
            tmp_wet = dict(wet_record.rec_headers.headers)
            
            #while (tmp_wet['WARC-Refers-To'] != tmp_warc['WARC-Record-ID'] and warc_record.rec_type != 'response'):
            while (tmp_wet['WARC-Refers-To'] != tmp_warc['WARC-Record-ID']):
                warc_record = iter_warc.__next__()
                warc_offset+=1
                #print(wet_offset, warc_offset)
                tmp_warc = dict(warc_record.rec_headers.headers)
            
            
            #print (tmp_wet['WARC-Refers-To'], tmp_warc['WARC-Record-ID'], warc_record.rec_type)
            
            record_dict = {'warc_offset':warc_offset,
                        'length':int(tmp_warc['Content-Length']),
                        'ip':tmp_warc['WARC-IP-Address'],
                        'id':tmp_warc['WARC-Record-ID'],
                        'URI':tmp_warc['WARC-Target-URI'],
                        'payload-digest': tmp_warc['WARC-Payload-Digest']
                          }
            #print (record_dict)
            record_dict['content_type'] = warc_record.http_headers.get_header('Content-Type')
            record_dict['text'] = wet_record.raw_stream.read()
            #print(record_dict['text'])
            #record_dict['text'] = parsed_text
                            
            #print(content_type)
        #print (record.rec_type, iter_warc.__next__().rec_type) 
        
        
        
print ("THE END")
#print ("Total parsed pages: ",parsed_pages,
#                  "\nTotal not parsed: ", not_parsed_pages)
#for key,value in not_parsed_types.items():
#    print ("\t",key,": ",value)

b = datetime.datetime.now()
print (b)
print (b-a)

2018-03-08 12:34:57.100783
THE END
2018-03-08 12:35:38.878316
0:00:41.777533


In [4]:
wet_data_file

{'date': '2018-01-24T14:12:43Z',
 'filename': 'CC-MAIN-20180116070444-20180116090444-00000.warc.wet.gz'}